In [1]:
import torch
from torch import nn
import pickle as pkl
import numpy as np
import MinkowskiEngine as ME

In [2]:
filehandler = open("data.pkl","rb")
dataset = pkl.load(filehandler)
filehandler.close()

In [3]:
torch.cuda.is_available()

True

In [4]:
dev = torch.device("cuda")
dev

device(type='cuda')

In [5]:
class Conv3DFeatureDetector(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        size_1 = 100
        size_2 = 200
        self.conv1 = nn.Sequential(
            ME.MinkowskiConvolution(1, 20, 
                      kernel_size=(kernel_size),
                      dimension=3,
                                   ),
            ME.MinkowskiConvolution(20, size_1, 
                      kernel_size=(kernel_size),
                      dimension=3,
                                   ),
            ME.MinkowskiTanh(),
#             nn.MaxPool3d((2, 2, 2)),
        )
        self.fc1 = ME.MinkowskiLinear(size_1, size_2)
        self.relu = ME.MinkowskiTanh()
        self.fc2 = ME.MinkowskiLinear(size_2, 1)
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = ME.MinkowskiFunctional.sigmoid(out)
        return out
    
# class Conv3DFeatureDetector(nn.Module):
#     def __init__(self, kernel_size):
#         super().__init__()
        
#         size_1 = 100
#         size_2 = 200
#         self.conv1 = nn.Sequential(
#             nn.Conv3d(1, size_1, kernel_size=(kernel_size, kernel_size, kernel_size), padding= (kernel_size - 1)//2),
#             nn.Tanh(),
# #             nn.MaxPool3d((2, 2, 2)),
#         )
#         self.fc1 = nn.Linear(size_1, size_2)
#         self.relu = nn.Tanh()
#         self.fc2 = nn.Linear(size_2, 1)
        
#     def forward(self, x):
#         out = self.conv1(x)
#         out = out.permute(0,2,3,4,1)
#         out = self.fc1(out)
#         out = self.relu(out)
#         out = self.fc2(out)
#         out = torch.sigmoid(out)
#         return out

In [6]:
X = np.vstack([x[None,None,:,:,:] for (x,y) in dataset])
X = X.astype(np.float32)
X = torch.from_numpy(X)

Y = np.vstack([y[None,:,:,:,None] for (x,y) in dataset])
Y = Y.astype(np.float32)
Y = torch.from_numpy(Y)
print(X.shape)
print(Y.shape)

X_train = X[:500]
Y_train = Y[:500]

def get_sparse_tensor_from_numpy(X, d=None):
    idxs = torch.where(X)
    num_nonzero_elements = idxs[0].shape[0]
    num_dimensions = len(idxs)
    coords = torch.vstack(idxs)
    coords = coords.type(torch.int32)
    coords = torch.transpose(coords, 0,1).contiguous()
    feat = torch.ones(coords.shape[0],1)
    if d is None:
        X = ME.SparseTensor(feat, coordinates=coords)
    else:
        X = ME.SparseTensor(feat, coordinates=coords, device=d)
    
    return X


torch.Size([1000, 1, 45, 45, 42])
torch.Size([1000, 45, 45, 42, 1])


In [7]:
X_train.mean()

tensor(0.0265)

In [8]:
# X.cuda()
model = Conv3DFeatureDetector(9)
model.to(dev)



Conv3DFeatureDetector(
  (conv1): Sequential(
    (0): MinkowskiConvolution(in=1, out=20, kernel_size=[9, 9, 9], stride=[1, 1, 1], dilation=[1, 1, 1])
    (1): MinkowskiConvolution(in=20, out=100, kernel_size=[9, 9, 9], stride=[1, 1, 1], dilation=[1, 1, 1])
    (2): MinkowskiTanh()
  )
  (fc1): MinkowskiLinear(in_features=100, out_features=200, bias=True)
  (relu): MinkowskiTanh()
  (fc2): MinkowskiLinear(in_features=200, out_features=1, bias=True)
)

In [9]:
learning_rate = 1.0
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [10]:
X_train[:10,0,:,:,:].shape

torch.Size([10, 45, 45, 42])

In [11]:
batch_size = 10
output = None
batch_y = None
batch_x = None
for epoch in range(1000):
#     permutation = torch.randperm(X_torch.size()[0])
    permutation = torch.arange(X_train.size()[0])
    
    total_loss = 0.0
    for i in range(0,X_train.size()[0], batch_size):
#         print(i)
        optimizer.zero_grad()

        indices = permutation[i:i+batch_size]
        batch_x, batch_y = X_train[indices], Y_train[indices]
        batch_y = batch_y.permute(0,4,1,2,3)
        batch_x = get_sparse_tensor_from_numpy(batch_x[:,0,:,:,:],d=dev)
        
        # in case you wanted a semi-full example
        outputs = model.forward(batch_x)
        
        outputs = outputs.dense(shape=batch_y.shape)[0].cpu()
        
        loss = (outputs - batch_y)**2
        loss[batch_y == 1.0] *= 100.0
        loss = loss.mean()
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    print("total loss: ", total_loss)

/home/nishadg/mcs/InverseGraphics/venv/lib/python3.8/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/nishadg/mcs/InverseGraphics/venv/lib/python3.8/site-packages/MinkowskiEngine/MinkowskiSparseTensor.py:525: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  coords = coords // tensor_stride


total loss:  1.2712226584553719
total loss:  1.190705068409443
total loss:  1.1751085286960006
total loss:  1.1595068145543337
total loss:  1.1388595951721072
total loss:  1.108972323127091
total loss:  1.0727880513295531
total loss:  1.0446845619007945
total loss:  1.0278302757069468
total loss:  1.0170968677848577
total loss:  1.0093665169551969
total loss:  1.003336863592267
total loss:  0.9984182761982083
total loss:  0.9942986089736223
total loss:  0.9907877948135138
total loss:  0.9877576166763902
total loss:  0.9851150820031762
total loss:  0.982789684087038
total loss:  0.9807261601090431
total loss:  0.9788802973926067
total loss:  0.9772161776199937
total loss:  0.9757043644785881
total loss:  0.9743206119164824
total loss:  0.9730449439957738
total loss:  0.971860914491117
total loss:  0.9707549829035997
total loss:  0.9697159808129072
total loss:  0.968734691850841
total loss:  0.9678035750985146
total loss:  0.9669163171201944
total loss:  0.9660677090287209
total loss:  0

KeyboardInterrupt: 

In [12]:
X_test = X[-50:]
Y_test = Y[-50:]

In [13]:
model_cpu = model.cpu()

In [14]:
pred_Y = model.forward(get_sparse_tensor_from_numpy(X_test[:,0,:,:,:]))

In [15]:
permuted_Y_test = Y_test.permute(0,4,1,2,3)
pred_Y = pred_Y.dense(shape=permuted_Y_test.shape)[0]
pred_Y = pred_Y.permute(0,2,3,4,1)
pred_Y.shape

torch.Size([50, 45, 45, 42, 1])

In [16]:
pred_Y[Y_test[-100:] == 1.0]

tensor([0.6641, 0.9367, 0.9761,  ..., 0.7604, 0.0000, 0.0000],
       grad_fn=<IndexBackward0>)

In [17]:
pred_Y[Y_test[-100:] == 0.0]

tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<IndexBackward0>)

In [18]:
with open("result.pkl","wb") as f:
    pkl.dump((X_test.cpu().detach().numpy(), pred_Y.cpu().detach().numpy()), f)